In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from google.colab import drive
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

kc_house_data = pd.read_csv('gdrive/My Drive/uwml/kc_house_data.csv', dtype=dtype_dict)
testing = pd.read_csv('gdrive/My Drive/uwml/wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('gdrive/My Drive/uwml/wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('gdrive/My Drive/uwml/wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [4]:
sales = kc_house_data.copy()
sales

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,0263000018,20140521T000000,360000.0,3.0,2.50,1530.0,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530.0,1509.0
21609,6600060120,20150223T000000,400000.0,4.0,2.50,2310.0,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830.0,7200.0
21610,1523300141,20140623T000000,402101.0,2.0,0.75,1020.0,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020.0,2007.0
21611,0291310100,20150116T000000,400000.0,3.0,2.50,1600.0,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410.0,1287.0


In [5]:
# Function to create regression inputs
def get_numpy_data(df, features, output):
  df['constant'] = 1
  x_list = ['constant'] + features
  x = df[x_list]
  x = x.to_numpy()
  y = df[output]
  y = y.to_numpy()
  return(x, y)

# RSS function
def get_residual_sum_of_squares(yhat, y):
  e = yhat - y
  RSS = np.sum(e * e)
  return(RSS)

# Predict outcomes
def predict_outcome(feature_matrix, weights):
  predictions = np.dot(feature_matrix, weights)
  return(predictions)

In [6]:
# Normalise features function
def normalize_features(features):
  norms = np.linalg.norm(features, axis=0)
  normalized_features = features / norms
  return (normalized_features, norms)

In [7]:
# Test norm function
test = np.array([[1.,2.,3.],[4.,5.,6.],[7.,8.,9.]])
nms = normalize_features(test)
print(test)
print(nms[0])
print(nms[1])

[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
[[0.12309149 0.20739034 0.26726124]
 [0.49236596 0.51847585 0.53452248]
 [0.86164044 0.82956136 0.80178373]]
[ 8.1240384   9.64365076 11.22497216]


In [8]:
# Step 9
x_pre_norm, y = get_numpy_data(sales, ['sqft_living', 'bedrooms'], 'price')
x, norms = normalize_features(x_pre_norm)
weights = [1.,4.,1.]
yhat = predict_outcome(x, weights)

In [9]:
ro_1 = sum(x[:,1] *(y - yhat + weights[1] * x[:,1]))
ro_2 = sum(x[:,2] *(y - yhat + weights[2] * x[:,2]))
print(ro_1)
print(ro_2)
#len(x[:,1])
#len(weights[1]*x[:,1])

87939470.82325152
80966698.66623905


In [10]:
# Step 10 / Q. 1
print('Min value : ',ro_2*2)
print('Max value : ',ro_1*1.9999)

Min value :  161933397.3324781
Max value :  175870147.69942072


In [11]:
# Step 11 / Q. 2
print('Min value : ',ro_2*2)
print('Max value : ',ro_1*2)

Min value :  161933397.3324781
Max value :  175878941.64650303


In [12]:
# Step 12
def lasso_coordinate_descent_step(i, feature_matrix, y, weights, l1_penalty):
  # compute prediction
  yhat = predict_outcome(feature_matrix, weights)
  # compute ro[i]
  ro_i = sum(feature_matrix[:,i] * (y - yhat + weights[i] * feature_matrix[:,i]))
  
  if i == 0: # intercept -- do not regularize
      new_weight_i = ro_i
  elif ro_i < -l1_penalty/2.:
      new_weight_i = ro_i + l1_penalty/2
  elif ro_i > l1_penalty/2.:
      new_weight_i = ro_i - l1_penalty/2
  else:
      new_weight_i = 0.
  
  return new_weight_i

In [13]:
# Test - should print 0.425558846691
import math
lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.4255588466910251

In [14]:
# Check answer at step 9
lasso_coordinate_descent_step(1, x, y, [1.,4.,1.], 1)

87939470.32325152

In [15]:
# Step 13
def lasso_cyclical_coordinate_descent(feature_matrix, y, initial_weights, l1_penalty, tolerance):
  
  feature_length = len(initial_weights)
  weights = np.array(initial_weights)
  old_weights = np.zeros(feature_length)
  weight_delta_array = np.zeros(feature_length)
  weight_delta = tolerance + 1

  counter = 0
  #for j in range(10):
  while weight_delta > tolerance:
    
    for i in range(feature_length):
      old_weights[i] = weights[i]
      weights[i] = lasso_coordinate_descent_step(i, feature_matrix, y, weights, l1_penalty)
      weight_delta_array[i] = old_weights[i] - weights[i]
    
    #print('new weights post loop:', weights) ###
    #print('start weights post loop:', old_weights) ###
    
    weight_delta_array = weights - old_weights
    #print('weight_delta_array: ',weight_delta_array) ###
    
    weight_delta = np.amax(abs(weight_delta_array))
    #print('weight_delta: ',weight_delta) ###

    counter += 1

  return weights

In [16]:
# Step 16 / Q.4
model_1_weights = lasso_cyclical_coordinate_descent(feature_matrix=x, y=y, initial_weights=np.array([0.,0.,0.]), l1_penalty=1e7, tolerance=1.0)
model_1_weights

array([21624997.95951872, 63157247.20788978,        0.        ])

In [17]:
# Step 15 / Q. 3

# Predict outcome
model_1_yhat = predict_outcome(x, model_1_weights)

# RSS function
model_1_rss = get_residual_sum_of_squares(model_1_yhat, y)

model_1_rss

1630492476715384.5

In [18]:
# Step 18
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [19]:
# Step 18
x_train_pre_norm, y_train = get_numpy_data(training, all_features, 'price')

In [20]:
# Step 18
x_train, norms_train = normalize_features(x_train_pre_norm)

In [21]:
#Step 19 / 20 - Q. 5
weights = np.zeros(x_train.shape[1])

weights1e7 = lasso_cyclical_coordinate_descent(feature_matrix=x_train, y=y_train, initial_weights=weights, l1_penalty=1e7, tolerance=1.0)

pd.DataFrame({'feature': ['intercept']+all_features, 'coefficients': weights1e7})

,feature,coefficients
0,intercept,2.386469e+07
1,bedrooms,0.000000e+00
2,bathrooms,0.000000e+00
3,sqft_living,3.049555e+07
4,sqft_lot,0.000000e+00
5,floors,0.000000e+00
6,waterfront,1.901634e+06
7,view,5.705765e+06
8,condition,0.000000e+00
9,grade,0.000000e+00


In [22]:
#Step 21 / 22 - Q.6
weights = np.zeros(x_train.shape[1])

weights1e8 = lasso_cyclical_coordinate_descent(feature_matrix=x_train, y=y_train, initial_weights=weights, l1_penalty=1e8, tolerance=1.0)

pd.DataFrame({'feature': ['intercept']+all_features, 'coefficients': weights1e8})

,feature,coefficients
0,intercept,5.362100e+07
1,bedrooms,0.000000e+00
2,bathrooms,0.000000e+00
3,sqft_living,0.000000e+00
4,sqft_lot,0.000000e+00
5,floors,0.000000e+00
6,waterfront,0.000000e+00
7,view,0.000000e+00
8,condition,0.000000e+00
9,grade,0.000000e+00


In [23]:
#Step 23 / 24 - Q.7
weights = np.zeros(x_train.shape[1])

weights1e4 = lasso_cyclical_coordinate_descent(feature_matrix=x_train, y=y_train, initial_weights=weights, l1_penalty=1e4, tolerance=5e5)

pd.DataFrame({'feature': ['intercept']+all_features, 'coefficients': weights1e4})

,feature,coefficients
0,intercept,5.748109e+07
1,bedrooms,-1.365263e+07
2,bathrooms,1.246271e+07
3,sqft_living,5.794279e+07
4,sqft_lot,-1.475770e+06
5,floors,-4.904548e+06
6,waterfront,5.349050e+06
7,view,5.845254e+06
8,condition,-4.160390e+05
9,grade,2.682275e+06


In [24]:
# Step 26
normalised_weights1e7 = weights1e7 / norms_train
normalised_weights1e8 = weights1e8 / norms_train
normalised_weights1e4 = weights1e4 / norms_train

normalised_weights1e7[3]

135.17651646138287

In [25]:
# Step 27
x_test_pre_norm, y_test = get_numpy_data(training, all_features, 'price')

# Predict outcome
weights1e7_yhat = predict_outcome(x_test_pre_norm, normalised_weights1e7)
weights1e8_yhat = predict_outcome(x_test_pre_norm, normalised_weights1e8)
weights1e4_yhat = predict_outcome(x_test_pre_norm, normalised_weights1e4)

# RSS function
weights1e7_rss = get_residual_sum_of_squares(weights1e7_yhat, y_test)
weights1e8_rss = get_residual_sum_of_squares(weights1e8_yhat, y_test)
weights1e4_rss = get_residual_sum_of_squares(weights1e4_yhat, y_test)

print(weights1e7_rss/1e12)
print(weights1e8_rss/1e12)
print(weights1e4_rss/1e12)

786.9186720135269
1405.842389371723
597.0388281003718
